# Central RL control notebook

In [1]:
import sys
from pymgrid import MicrogridGenerator as mg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.display import display,clear_output
import pandas as pd
from Storage import StorageSuite as st
from pathlib import Path
from math import e
from time import sleep
from DQNEnv import DQAgent
import pickle as pkl
import random

Time span constants

In [2]:
HOUR = 4 # 15 Min intervals in an hour
DAY = 96 # Number of 15 Min intervals in day
MONTH = 2_924 # Number of 15 Min intervals in month
YEAR =35_088 # Number of 15 Min intervals in year (non-leap)
ZERO = 10**-5

### Generate Microgrids

In [3]:
n_grids = 2
grid_list = []

for i in range(n_grids):
    grid_list.append(st(r'C:\Users\thesu\Desktop\Research 2022 Microgrid\Research code repository\Micro-Grid-ML-algo\data\dummy_data_v1.csv',load=np.random.randint(1E5,6E5)))
mg_env = mg.MicrogridGenerator(storage_suite_list=grid_list)
mg_env.generate_microgrid(verbose= True, interpolate= True)

c:\Users\thesu\anaconda3\lib\site-packages\pymgrid\MicrogridGenerator.py:518: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,349342,349342,0.000025,0.000003,0.000025,216592.04,1,698684,698684,3,0.510391,0.995274,0.087814,388158,0.05,0.9,0.00001,0.000005
1,377869,377869,0.000025,0.000003,0.000025,226721.40,1,755738,755738,3,1.933146,0.573415,0.059300,419855,0.05,0.9,0.00001,0.000005


In [4]:
mg0 = mg_env.microgrids[0]
mg1 = mg_env.microgrids[1]

In [5]:
mg_env.print_mg_parameters()

c:\Users\thesu\anaconda3\lib\site-packages\pymgrid\MicrogridGenerator.py:518: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,349342,349342,0.000025,0.000003,0.000025,216592.04,1,698684,698684,3,0.510391,0.995274,0.087814,388158,0.05,0.9,0.00001,0.000005
1,377869,377869,0.000025,0.000003,0.000025,226721.40,1,755738,755738,3,1.933146,0.573415,0.059300,419855,0.05,0.9,0.00001,0.000005


In [6]:
# li_ion, flow, flywheel = mg0.ss.unpack()
# pv =                            round(mg0.pv)
# load =                          round(mg0.load)
# net_load =                      load-pv
# status =                        mg0.grid.status
# # overgen_net_load =            pv-load
# li_ion_capa_to_charge =         li_ion.capa_to_charge
# li_ion_capa_to_discharge =      li_ion.capa_to_discharge
# flow_capa_to_charge =           flow.capa_to_charge
# flow_capa_to_discharge =        flow.capa_to_discharge
# flywheel_capa_to_charge =       flywheel.capa_to_charge
# flywheel_capa_to_discharge =    flywheel.capa_to_discharge

# li_ion_charge =                 max(0,min(-net_load,li_ion_capa_to_charge ,li_ion.power))
# li_ion_discharge =              max(0,min(net_load,li_ion_capa_to_discharge,li_ion.power))

# flow_charge =                   max(0,min(-net_load,flow_capa_to_charge ,flow.power))
# flow_discharge =                max(0,min(net_load,flow_capa_to_discharge,flow.power))

# flywheel_charge=                max(0,min(-net_load,flywheel_capa_to_charge ,flywheel.power))
# flywheel_discharge =            max(0,min(net_load,flywheel_capa_to_discharge,flywheel.power))

# li_ion_soc =                 abs(li_ion.soc)
# flow_soc =                   abs(flow.soc)
# flywheel_soc =               abs(flywheel.soc)

# capa_to_genset = mg0.genset.rated_power * mg0.genset.p_max
# p_genset = max(0, min(net_load, capa_to_genset))


# # soc_list = {'li_ion_soc': li_ion_soc, 'flow_soc': flow_soc, 'flywheel_soc': flywheel_soc}
# # soc_list = sorted(soc_list, key=lambda x: (soc_list[x]))

# control_dict = {    'pv_consummed': min(pv,load),
#                     'li_charge': 0,
#                     'li_discharge': li_ion_discharge,
#                     'flow_charge': 0,
#                     'flow_discharge': flow_discharge,
#                     'flywheel_charge': 0,
#                     'flywheel_discharge': flywheel_discharge,
#                     'grid_import': 0,
#                     'grid_export': li_ion_discharge+flow_discharge+flywheel_discharge,
#                     'genset': 0
#                 }
# print(min(pv,load))
# new_state,reward, done, = mg0.run(control_dict)
# # print(len(new_state))
# li_ion.soc = li_ion.MAX_SOC
# flow.soc = flow.MAX_SOC
# flywheel.soc = flywheel.MAX_SOC
# mg0.reset()
# print(p_genset)
# print(reward)
# print(' ')


Now I define my agent function

In [7]:
def train_new_network(env: object, n_episodes: int, nb_actions: int, horizon: int) -> object:
    '''Used to train the deep Q-learning model and store a deep Q model
        For path use \\ or r'' to avoid UNICODE errors '''
    env.set_horizon(horizon = horizon) # Sets the Horizon
    agent = DQAgent(learning_rate=0.05, gamma=0.90, batch_size=96, 
                    state_len=len(env.reset()), 
                    n_actions = nb_actions,
                    mem_size = 1000000,
                    min_memory_for_training=1000, epsilon=1, epsilon_dec=0.999,
                    epsilon_min = 0.02)
    #main training loop
    for episode in range(n_episodes):
        state = env.reset()                              
        score = 0                                                                   
        done = 0                                                             
        while not done:                                                                                         
            action_select = agent.choose_action(state)
            action = env.actions_agent(action = action_select) 
            new_state,reward, done, = env.run(action)                             
            score+=(-reward)
            agent.store_transition(state, action_select, -reward, new_state, done)       
            agent.learn()                                                           
            state = new_state                                                       
            value_print=f"\rEpisode: {episode} Progress " + str(round(((env._tracking_timestep)*100)/(env.horizon),1))
            sys.stdout.write(value_print)
            sys.stdout.flush()
    env.reset()
    return agent

In [8]:
def test_grid(env: object, horizon: int, agent: object) -> float:
    ''' Manages the grid based on a trained neural net model '''
    env.set_horizon(horizon = horizon) # Sets the Horizon

    state = env.reset()                                                       
    score = 0                                                                   
    done = 0
    score_record = []
    action_record = []
    #main testing loop                                                            
    while not done:                                                                                         
        action_select = agent.choose_action(state)
        action = env.actions_agent(action = action_select)
        new_state,reward, done, = env.run(action)
        score+=(-reward)
        score_record.append(score)
        action_record.append(action_select)
        state = new_state
        # value_print="\rProgress " + str(round(((env._tracking_timestep)*100)/(env.horizon),1)) +" %"
        sys.stdout.write(str(f'\r{score}'))
        sys.stdout.flush()
    env.reset()
    return score, score_record, action_record

In [9]:
def randomize(mg=mg_env.microgrids[0], nb_actions = 8, horizon = DAY):
    ''' Manages the grid with random actions. Used to compare to deep Q-learning algorithm '''
    env = mg
    env.set_horizon(horizon = horizon) # Sets the Horizon
    li_ion,flow,flywheel = mg.ss.unpack()
    state = env.reset()                                                       
    score = 0                                                                   
    done = 0
    while not done:                                                                                         
        action_select = np.random.randint(0,8)
        action = env.actions_agent(action = action_select) 
        new_state, reward, done, = env.run(action)                          
        score+=(-reward)
        sys.stdout.write(str(f'\r{score}'))
        sys.stdout.flush()                                                                                                               
        state = new_state                                                       
    env.reset()
    return score

In [10]:
trained_agent = train_new_network(env=mg1, n_episodes=100, nb_actions=5, horizon = DAY*7)

Episode: 4 Progress 88.10

KeyboardInterrupt: 

In [ ]:
with open(r'C:\Users\thesu\Desktop\New_Agent_7_22_2022.pkl', 'wb') as f:  
     pkl.dump(obj = trained_agent, file=f)

Trained Network Storage

with open(r'C:\Users\thesu\Desktop\Trained Agent 7_3_2022.pkl', 'wb') as f:  
    pkl.dump(obj =trained_agent, file=f)

Object Loading

with open(r'C:\Users\thesu\Desktop\Trained Agent 7_3_2022.pkl', 'rb') as f:  
    loaded_agent = pkl.load(f)

In [ ]:
score, test_score_record, action_record = test_grid(env = mg1, horizon = YEAR, agent = trained_agent)

In [ ]:
counter = 0
for i in range(len(action_record)):
    if action_record[i] == 2:
        counter += 1
print(counter)

In [ ]:
plt.hist(action_record)
plt.show()

In [ ]:
r_score = randomize(mg=mg1, nb_actions = 5, horizon = YEAR)


In [ ]:
print(r_score)

In [ ]:
# plt.plot(r_score_record)
# plt.show()

In [ ]:
percentage = ((score)/(r_score)-1)*100

In [ ]:
print(str(round(percentage,2))+" %")

15 mins = 29.32%

In [ ]:
score2 = -score